In [5]:
import requests
import math
import time
import json
import random
import csv
import datetime

from dateutil.relativedelta import relativedelta
import pandas as pd
from pprint import pprint
from tqdm import tqdm
from typing import Tuple

In [21]:
with open("augmented_players.json", "r") as f:
    players = json.loads(f.read())

In [44]:
min_mins_requirement = 300
min_games = 8

player_map = {}

for p in players:
    player_map[p["name"]["display"]] = p
    
print(len(filtered_players))

## When choosing a random player, restrict the subset to players people would actually know
filtered_players = [p for p in players if p["appearances"] >= min_games and p["mins_played"] > min_mins_requirement and p["position"] != "G"]
def pick_random_player():
    return random.choice(list(filtered_players))

347


In [4]:
## Initialize the Country -> Continent Mapping
confederation_mapping = {}
with open("fifa.csv", "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    for row in reader:
        confederation_mapping[row[0]] = row[1]
        
print(confederation_mapping["England"])
    

UEFA


In [16]:
## Initialize EPL Table
with open("epl_table.json", "r") as f:
    epl_table = json.loads(f.read())
    standings = [t["team"]["name"] for t in epl_table]
    pprint(standings)
    
def get_epl_table_diff(team1, team2):
    position1 = standings.index(team1)
    position2 = standings.index(team2)
    return (position1 - position2)

['Manchester City',
 'Liverpool',
 'Chelsea',
 'Arsenal',
 'Tottenham Hotspur',
 'Manchester United',
 'West Ham United',
 'Wolverhampton Wanderers',
 'Aston Villa',
 'Leicester City',
 'Southampton',
 'Crystal Palace',
 'Brighton and Hove Albion',
 'Newcastle United',
 'Brentford',
 'Leeds United',
 'Everton',
 'Watford',
 'Burnley',
 'Norwich City']


In [46]:
class TrieNode(object):
    """
    Our trie node implementation. Store a search word as a pointer to the full word
    """
    
    def __init__(self, char: str, value: str):
        self.char = char
        self.children = []
        # Is it the last character of the word.`
        self.word_finished = False
        # How many times this character appeared in the addition process
        self.counter = 1
        self.value = value
    

def add(root, search_word: str, value: str):
    """
    Adding a word in the trie structure
    """
    node = root
    for char in search_word:
        found_in_child = False
        # Search for the character in the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found it, increase the counter by 1 to keep track that another
                # word has it as well
                child.counter += 1
                # And point the node to the child that contains this char
                node = child
                found_in_child = True
                break
        # We did not find it so add a new chlid
        if not found_in_child:
            new_node = TrieNode(char, "")
            node.children.append(new_node)
            # And then point node to the new child
            node = new_node
    # Everything finished. Mark it as the end of a word.
    node.word_finished = True
    node.value = value


def find_prefix(root, prefix: str):
    """
    Check and return 
      1. If the prefix exsists in any of the words we added so far
      2. If yes then how may words actually have the prefix
    """
    node = root
    # If the root node has no children, then return False.
    # Because it means we are trying to search in an empty trie
    if not root.children:
        return []
    for char in prefix:
        char_not_found = True
        # Search through all the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found the char existing in the child.
                char_not_found = False
                # Assign node as the child containing the char and break
                node = child
                break
        # Return False anyway when we did not find a char.
        if char_not_found:
            return []
    
    ## Now that we have the node, let's get all leaf nodes
    res = []
    ## TO DO: If exact match, ignore partial matches --> Remove once dropdown is added
    if node.word_finished:
        return [node.value]
    
    def dfs(nested_prefix, node):
        if node.word_finished:
            res.append(node.value)
        for child in node.children:
            dfs(nested_prefix + child.char, child)

    dfs(prefix, node)
    return res
            

## Initialize the Trie Tree - 1 For display Name, 1 for Last Name
root = TrieNode('*', '*')
for name in [p["name"] for p in players]:
    add(root, name["display"], name["display"])
    

last_name_root = TrieNode("*", "*")
for name in [p["name"] for p in players]:
    add(last_name_root, name["last"], name["display"])
    

def search_ahead(prefix):
    using_all = find_prefix(root, prefix)
    using_last = find_prefix(last_name_root, prefix)
    return list(set(using_all + using_last))


print(search_ahead("Harry"))
print(search_ahead("Magui"))

['Harry Winks', 'Harry Lewis', 'Harry Maguire', 'Harry Kane', 'Harry Tyrer']
['Harry Maguire']


In [36]:
def get_name(player):
    return player["name"]

def get_goals(player):
    return player.get("goals",0)

def get_assists(player):
    return player.get("assists", 0)

def get_appearances(player):
    return player["appearances"]

def get_mins(player):
    return player["mins_played"]

def get_team(player):
    return player["team"]

def get_country(player):
    return player["nation"]

def get_age(player):
    date = datetime.datetime.fromtimestamp(player["dob"] / 1000)
    return relativedelta(datetime.datetime.now(), date).years

def get_position(player):
    return player["position"]

def get_jersey(player):
    return player["number"]



CORRECT = "✓"
LOWER = "↓"
HIGHER = "↑"
WRONG = "X"
CLOSE = "Close: ✓"
CONFEDERATION = "Confederation: ✓"


def main():
    player = pick_random_player()
    
    guesses = []
    max_guesses = 6
    
    
    guess_history = [["Name", "Team","Position","Goals", "Assists", "Appearances", "Country", "Age", "Jersey"]]
    
    try:
        while (len(guesses) < max_guesses):
            player_name_guess = ""
            while player_name_guess == "":
                input_name = input("PLAYER NAME: ")

                if input_name == "BREAK":
                    raise ValueError('Breaking out of the program!')


                valid_names = search_ahead(input_name)
                if len(valid_names) == 1:
                    name = valid_names[0]
                    if name in [p["name"]["display"] for p in guesses]:
                        print("DUMBASS! You already guessed that")
                    else:
                        player_name_guess = name
                else:
                    print("VAlID NAMES: " + ", ".join(valid_names))

            guess = player_map[player_name_guess]
            guess_name = get_name(guess)
            guess_goals = get_goals(guess)
            guess_assists = get_assists(guess)
            guess_mins_played = get_mins(guess)
            guess_apps = get_appearances(guess)
            guess_country = get_country(guess)["country"]
            guess_team = get_team(guess)
            guess_age = get_age(guess)
            guess_position = get_position(guess)
            guess_jersey = get_jersey(guess)

            ## Exact Matches
            def get_exact_match_code(player_v, guess_v):
                if player_v == guess_v:
                    return CORRECT
                else:
                    return WRONG

            def get_position_code():
                player_v = get_position(player)
                guess_v = get_position(guess)
                return get_exact_match_code(player_v, guess_v)


            # Comparison Matches with "Closeness"
            def get_comparison_code(player_v, guess_v, offset):
                comp = player_v - guess_v
                if player_v == guess_v:
                    return CORRECT
                elif comp > 0 and comp <= offset:
                    return HIGHER
                elif comp < 0 and comp >= -offset:
                    return LOWER
                else:
                    return WRONG

            def get_goals_code():
                player_v = get_goals(player)
                guess_v = get_goals(guess)
                return get_comparison_code(player_v, guess_v, 4)

            def get_assists_code():
                player_v = get_assists(player)
                guess_v = get_assists(guess)
                return get_comparison_code(player_v, guess_v, 2)


            def get_mins_code():
                player_v = get_mins(player)
                guess_v = get_mins(guess)
                return get_comparison_code(player_v, guess_v, 500)

            def get_age_code():
                player_v = get_age(player)
                guess_v = get_age(guess)
                return get_comparison_code(player_v, guess_v, 2)


            def get_appearances_code():
                player_v = get_appearances(player)
                guess_v = get_appearances(guess)
                return get_comparison_code(player_v, guess_v, 5)
            
            
            def get_jersey_code():
                player_v = get_jersey(player)
                guess_v = get_jersey(guess)
                return get_comparison_code(player_v, guess_v, 2)
            
            def get_country_code():
                player_v = get_country(player)["country"]
                guess_v = get_country(guess)["country"]
                if player_v == guess_v:
                    return CORRECT
                elif confederation_mapping.get(player_v, "a") == confederation_mapping.get(guess_v, "b"):
                    return CONFEDERATION
                else:
                    return WRONG
                
            def get_team_code():
                player_v = get_team(player)
                guess_v = get_team(guess)
                if player_v == guess_v:
                    return CORRECT
                elif abs(get_epl_table_diff(player_v, guess_v)) <= 2:
                    return CLOSE
                else:
                    return WRONG


            if guess["name"] == player["name"]:
                print(player["name"]["display"] + " is correct! YOU WIN!!")
                break
            else:
                guess_data = [guess["name"]["display"], guess_team, guess_position, guess_goals, guess_assists, guess_apps, guess_country, guess_age, guess_jersey]
                guess_hints = ["", get_team_code(), get_position_code(), get_goals_code(), get_assists_code(), get_appearances_code(), get_country_code(), get_age_code(), get_jersey_code()]
                guess_history.append(guess_data)
                guess_history.append(guess_hints)
                
                l1, l2 = len(guess_history), len(guess_history[0])
                pd.set_option('expand_frame_repr', False)
                print(pd.DataFrame(guess_history, index=['']*l1, columns=['']*l2))
                
                guesses.append(guess)
                
                
    except ValueError as err:
        print(err.args)
    
    if len(guesses) >= max_guesses:
        print("YOU LOSE!")
        print("Correct answer was: " + player["name"]["display"])
        print(player)

In [48]:
main()

PLAYER NAME:  Ziyech


                                                                                     
          Name      Team  Position  Goals  Assists  Appearances  Country  Age  Jersey
  Hakim Ziyech   Chelsea         M      4        2           15  Morocco   29      22
                Close: ✓         ✓      X        X            X        X    X       X


PLAYER NAME:  Alcan


VAlID NAMES: 


PLAYER NAME:  Thiago


...

VAlID NAMES: 


PLAYER NAME:  De Br


                                                                                                        
             Name             Team  Position  Goals  Assists  Appearances           Country  Age  Jersey
     Hakim Ziyech          Chelsea         M      4        2           15           Morocco   29      22
                          Close: ✓         ✓      X        X            X                 X    X       X
           Thiago        Liverpool         M      1        2           17             Spain   30       6
                          Close: ✓         ✓      X        X            X  Confederation: ✓    X       ↑
  Kevin De Bruyne  Manchester City         M      9        3           22           Belgium   30      17
                                 X         ✓      ✓        ↑            X  Confederation: ✓    X       X


PLAYER NAME:  Saka


Bukayo Saka is correct! YOU WIN!!
